In [1]:
%matplotlib inline
# %load_ext autoreload
# %autoreload 1 # Always reload before execution only modules imported with '%aimport'
# %aimport pipeline
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import random
import os
import sys
import itertools
sys.path.append('src/')
import nn
import process_data
import nibabel as nib
from math import floor, ceil

from sklearn.metrics import confusion_matrix
import scipy.sparse
from scipy.misc import imrotate, imresize
from scipy.ndimage.filters import gaussian_filter
from scipy.ndimage import rotate
from skimage import exposure
from skimage.io import imread, imsave
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from tensorflow.python.client import device_lib
local_device_protos = device_lib.list_local_devices()
print(local_device_protos)

import pipeline
import Unet

/home/jessica/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18034274451089266606
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 11970700903
locality {
  bus_id: 1
}
incarnation: 13485255748377308177
physical_device_desc: "device: 0, name: TITAN Xp, pci bus id: 0000:02:00.0"
]


# Using this notebook

This notebook is meant to be a demo and documentation for using the models and generating predictions. Feel free to make copies for your own use but please do not commit your changes to it (unless there's an error, in which case submit a pull request).

# Initialize the Tensorflow session and a dummy model

This needs to be done before any operation involving training or generating segmentations.

In [2]:
tf.reset_default_graph()
sess = tf.Session()
model = Unet.Unet(0, 0.5, 0.5) # Arbitrary initialization
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

# Load a pretrained model

To do this you need to specify the directory where the models are held as well as the saved model name, which should match both the name of the subfolder that holds the model as well as the `[model_name].data, [model_name].meta`, and `[model_name].index` files. The data file might have something on the end of its extension, like `[model_name].data-00000-of-00001`. That's fine.

Note that `models_dir` should not be the highest-level directory containing folders for each model architecture, e.g. `/media/jessica/Storage/models/`, but should be one of the folders corresponding to a particular model architecture that in turn holds different trained versions. In kind-of picture form:


In the above picture, don't use `models` as `models_dir`. You could use the absolute path to `u-net_v1-0` or `u-net_v2-0` and then the model name (same as folder name) of one of the pretrained versions, e.g. `pretrained_version_1`.

The below example will work on the main office machine but won't on your own unless you download the model and put it on your machine and set the directories right.

### Set directories

In [3]:
models_dir = '/media/jessica/Storage/models/u-net_v1-0'
model_name = '30_deg_training_sorted_inputs'

### Load model

In [4]:
pipeline.load_model(models_dir, model_name, saver, sess)

INFO:tensorflow:Restoring parameters from /media/jessica/Storage/models/u-net_v1-0/30_deg_training_sorted_inputs/30_deg_training_sorted_inputs


# Predict many segmentations

The best function for generating more than one segmentation at once is the `predict_all_segs` function in `pipeline.py`. With this function you can easily generate predictions for every single preprocessed scan and NIfTI pair you have (or in general as many as you want). To use this, the Tensorflow session and model need to be loaded as above and then three directories need to be specified: the directory containing separate folders each holding preprocessed scan data, the directory where all the resulting segmentations will be placed, and the directory holding the original `.nii` files. Note for this last directory that you will most likely want to use `/media/jessica/Storage/allrawfillednifti` rather than `/media/jessica/Storage/allrawfnifti`. The difference is the former holds the reconstructed `.nii` files where the deltoid is filled in. Additionally, the scan dimensions will match the generated predictions, so you can open the predictions over these filled `.nii` files. Once these are specified, all you need to do is call the function with the directories as well the Tensorflow model and session arguments.

### Set directories

In [ ]:
to_be_segmented_dir = "/media/jessica/Storage/preprocesseddata"
new_segs_location_dir = "/media/jessica/Storage/30_deg_training_pred_dump"
orig_nii_dir = "/media/jessica/Storage/allrawfillednifti"

### Generate segmentations

In [ ]:
pipeline.predict_all_segs(to_be_segmented_dir, new_segs_location_dir, orig_nii_dir, model, sess)

# Predict a single segmentation (or control intermediate steps)

In general `predict_all_segs` is the best way to make segmentations, even if you just want to generate one segmentation. If you're manually loading and modifying data or otherwise want access to intermediate steps then you can do that as well, but it will be more inconvenient, especially if you want to do this for more than one scan.

To load the data from a single scan, use `load_data` from `pipeline.py`. You will need to provide the path to the directory which directly holds the `.npz` and `.png` files of the scan you want to segment. Note you can set the optional flag `encode_segs` (True by default) to determine whether you want to one-hot-encode the ground truth labels. If you are just generating a segmentation this should probably be set to False because encoding can take a few minutes. If you have no label files it doesn't matter. Note the outputs from this function are lists whereas future functions expect Numpy arrays, so you will have to convert them before passing them to prediction functions. It's likely best for now to leave the `height` and `width` arguments as their default values to ensure compatibility with existing scans.

### Set directory

In [5]:
processed_data_dir = "/media/jessica/Storage/preprocesseddata/trial8_30_fs"

### Load data

In [6]:
raw_imgs_lst, seg_imgs_lst = pipeline.load_data(processed_data_dir, encode_segs=False)

====
/media/jessica/Storage/preprocesseddata/trial8_30_fs
====


### See output details

In [7]:
print(len(raw_imgs_lst), len(seg_imgs_lst), sep=', ')
print(type(raw_imgs_lst), type(seg_imgs_lst), sep=', ')
print(type(raw_imgs_lst[0]), type(seg_imgs_lst[0]), sep=', ')
print(raw_imgs_lst[0].shape, seg_imgs_lst[0].shape, sep=', ')

996, 996
<class 'list'>, <class 'list'>
<class 'numpy.ndarray'>, <class 'numpy.ndarray'>
(512, 512), (512, 512)


Having loaded the data, you can proceed to do whatever inspection, visualization, or processing you want. Once you want to predict, though, you will need to convert the lists to Numpy arrays. You can then pass the array into `predict_whole_seg`.

### Convert to array and predict

In [8]:
raw_imgs_arr = np.asarray(raw_imgs_lst)
print("Shape of new array:", raw_imgs_arr.shape)
predicted_seg = pipeline.predict_whole_seg(raw_imgs_arr, model, sess)
print("Shape of resulting array (should be same as before):", predicted_seg.shape)


Shape of new array: (996, 512, 512)
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214,

Something to note is that the prediction functions actually want arrays to have a channel dimension (which for our purposes is of size 1; for RGB it would be 3). This is handled in the prediction function (see the call to `np.expand_dims`) but it might be important to know if you start directly calling some of those functions, like `predict_image` (which doesn't check this).

Once you have the segmentation you can play with it some more or save it as a NIfTI file by calling `save_arr_as_nifti`. Since you're calling it manually you will need to explicitly provide the name of the original NIfTI file (including the file extension) as well as the name to save the NIfTI as (also including file extension). You will also need the path to the directory where the NIfTI is stored and the path to the directory where you want to save the NIfTI. Again, note that the original NIfTI files are the filled versions.

### Set directories

In [9]:
orig_nifti_name = "trial8_30_fs_proc_filled_volume.nii"
new_nifti_name = "demo.nii"
orig_nii_dir = "/media/jessica/Storage/allrawfillednifti"
save_dir = "/media/jessica/Storage/demo"

### Save predicted segmentation as NIfTI

In [10]:
pipeline.save_arr_as_nifti(predicted_seg, orig_nifti_name, new_nifti_name, orig_nii_dir, save_dir)